<a href="https://colab.research.google.com/github/mjbernalv/trabajo2-si7006-252-perfilamiento/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Procesamiento de datos a trusted </center>

In [ ]:
!apt-get update
!apt-get install openjdk-11-jdk -y

## Librerías

In [ ]:
import re
import glob
import shutil
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F, Window
from pyspark.sql.types import IntegerType, DoubleType

## Funciones

In [ ]:
def limpiar_nombre(nombre):
    if nombre is None:
        return "Desconocido"
    # Reemplazar cualquier carácter que no sea letra o número por "_"
    return re.sub(r'[^A-Za-z0-9_]', '_', nombre.strip())

## Lectura de datos

In [24]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder \
    .appName("raw-to-trusted-spark") \
    .getOrCreate()

print("Spark iniciado:", spark)

Spark iniciado: <pyspark.sql.session.SparkSession object at 0x7e982c5bfbc0>


In [ ]:
ruta_base = "/content/drive/MyDrive/Dataset PI/"

detalle_poliza = ruta_base + "Raw/detalle_poliza.csv"
detalle_producto = ruta_base + "Raw/detalle_producto.csv"
detalle_cliente = ruta_base + "Raw/detalle_cliente.csv"

# Leer con inferencia de esquema y header
df_poliza = spark.read.option("header", "true").option("inferSchema", "true").csv(detalle_poliza)
df_producto = spark.read.option("header", "true").option("inferSchema", "true").csv(detalle_producto)
df_clientes = spark.read.option("header", "true").option("inferSchema", "true").csv(detalle_cliente)

# Mostrar confirmación
print("Poliza:", df_poliza.count(), "filas,", len(df_poliza.columns), "columnas")
print("Producto:", df_producto.count(), "filas,", len(df_producto.columns), "columnas")
print("Clientes:", df_clientes.count(), "filas,", len(df_clientes.columns), "columnas")

Poliza: 226254 filas, 28 columnas
Producto: 562 filas, 11 columnas
Clientes: 253062 filas, 8 columnas


## Transformaciones

### Unión tablas

In [ ]:
# Unión de poliza con clientes
df_poliza_clientes = df_poliza.join(
    df_clientes,
    on="codCliente",  # clave común
    how="left"
)

# Unión del resultado anterior con producto
df_final = df_poliza_clientes.join(
    df_producto,
    on="codProducto",  # clave común
    how="left"
)

# Mostrar resultado
print("Filas finales:", df_final.count())
print("Columnas finales:", len(df_final.columns))
df_final.show(5, truncate=False)


Filas finales: 226254
Columnas finales: 45
+-----------+----------+---------+-------------------+-----------+----------+-------------------+----------+-----------------+------+----------+-------------+---------------------+-------------------+----------------+-----------+------------------+----------+-----------+---------------+--------+----------+------------------------+--------------------+----------------+----------------------------+------------------+------------------+--------------------------+----------------+-------------------+----+------------------------------+------------+------------+------------------+-------+----------+-------------+-------------+--------------------------------+-------------------+----------+----------+------------------+
|codProducto|codCliente|codPoliza|estadoProcesamiento|fechaInicio|fechaFin  |origenProcesamiento|estadoDato|formaPagoVigencia|IVA   |valorTotal|IVA_Periodico|Valor_Total_Periodico|fechaInicioVigencia|fechaFinVigencia|estado     |Esta

In [ ]:
# Ruta donde guardaremos la tabla unida
ruta_salida = "/content/drive/MyDrive/Dataset PI/tabla_merge.csv"

# Guardar el dataframe como CSV con encabezado
df_final.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv(ruta_salida)

print(" Tabla final guardada en:", ruta_salida)

# Buscar el archivo generado dentro de la carpeta
archivo_generado = glob.glob(ruta_salida + "/part-*.csv")[0]

# Moverlo a la misma carpeta con el nombre tabla_final.csv
ruta_final = "/content/drive/MyDrive/Dataset PI/tabla_merge_unida.csv"
shutil.move(archivo_generado, ruta_final)

# Eliminar la carpeta vacía
shutil.rmtree(ruta_salida)

### Eliminación columnas no relevantes

In [28]:
columnas_a_eliminar = [
    "codProducto",
    "estadoProcesamiento",
    "origenProcesamiento",
    "estadoDato",
    "IVA",
    "IVA_Periodico",
    "fechaInicio",
    "fechaFin",
    "EstadoRenovaciones",
    "tipoPoliza",
    "Fecha",
    "valorTotalSinIva",
    "Valor_Total_Periodico_SinIva",
    "TasaIVA",
    "TasaIVA_Periodico",
    "fechaNacCliente",
    "FechaCumple",
    "nomTipoDocumento",
    "Tiene poliza",
    "codRamo",
    "seRenuevaRamo",
    "nomGrupo",
    "nomCompania",
    "GrupoRamo",
    "nomRamo",
    "NegociosEspeciales",
    "aseguradora",
    "tomador"
    "nomProducto"
]
# Eliminar columnas (solo si existen en el dataframe)
df_final = df_final.drop(*[c for c in columnas_a_eliminar if c in df_final.columns])

print("Columnas eliminadas correctamente")
print("Número de columnas restantes:", len(df_final.columns))


Columnas eliminadas correctamente
Número de columnas restantes: 19


### Generar nuevas categorías de ramos

In [ ]:
df_final = df_final.withColumn(
    "Ramo",
    F.when(F.col("Ramo").isin(
        "Salud Familiar", "Enfermedades Graves", "Salud Colectivo",
        "Asistencia Medica", "Medicina Prepagada"
    ), "Salud")
    .when(F.col("Ramo").isin(
        "Vida Individual", "Vida Grupo", "Educacion", "Pension",
        "Accidentes Personales", "Juvenil", "Exequial"
    ), "Vida")
    .when(F.col("Ramo").isin(
        "Automoviles", "Soat"
    ), "Autos")
    .when(F.col("Ramo").isin(
        "Cumplimiento", "Arrendamiento"
    ), "Cumplimiento")
    .when(F.col("Ramo").isin(
        "Incendio", "Responsabilidad Civil", "Hogar", "Transporte Mercancia",
        "Transporte Valores", "Maquinaria Y Equipo", "Manejo", "Copropiedad",
        "Montaje", "Construcción", "Navegacion",
        "R.C. Parqueaderos Y Talleres", "Agricola"
    ), "Patrimoniales")
    .otherwise("Otros")
)

### Reemplazar caracteres 

In [ ]:
# Reemplazamos espacios y caracteres raros por guiones bajos
for old_name in df_final.columns:
    new_name = old_name.strip().replace(" ", "_").replace("-", "_")
    df_final = df_final.withColumnRenamed(old_name, new_name)

### Validar tipos de datos

In [ ]:
df_final = df_final.withColumn("Edad", col("Edad").cast(IntegerType()))
df_final = df_final.withColumn("valorTotal", col("valorTotal").cast(DoubleType()))
df_final = df_final.withColumn("fechaInicioVigencia", F.to_date("fechaInicioVigencia", "dd/MM/yyyy"))

### Filtros con reglas de negocio

In [ ]:
# Eliminar filas donde valorTotal sea 0 o nulo
df_final = df_final.filter((F.col("valorTotal") > 0) & (F.col("valorTotal").isNotNull()))

### Eliminar duplicados

In [ ]:
# Eliminar duplicados por las columnas clave
df_final = df_final.dropDuplicates(["codCliente", "codPoliza", "fechaInicioVigencia", "fechaFinVigencia", "Ramo"])

### Crear columna de orden

In [ ]:
# Crear ventana por cliente y póliza, ordenada por fecha de inicio
window_spec = Window.partitionBy("codCliente", "codPoliza").orderBy(F.asc("fechaInicioVigencia"))

# Agregar columna de orden (opcional, para validar el orden)
df_final = df_final.withColumn("orden_vigencia", F.row_number().over(window_spec))

df_final = df_final.filter(F.col("orden_vigencia") == 1)

## Verificar datos

In [ ]:
# Reordenar para verificar
df_final = df_final.orderBy("codCliente", "codPoliza", "fechaInicioVigencia")

print("Número de registros:", df_final.count())

In [ ]:
df_final.show( truncate=False)

## Guardar datos

In [ ]:
# Ruta donde guardaremos la tabla unida
ruta_salida = "/content/drive/MyDrive/Dataset PI/trusted/trusted_temp.csv"

# Guardar el dataframe como CSV con encabezado
df_final.coalesce(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv(ruta_salida)

print(" Tabla final guardada en:", ruta_salida)

# Buscar el archivo generado dentro de la carpeta
archivo_generado = glob.glob(ruta_salida + "/part-*.csv")[0]

# Moverlo a la misma carpeta con el nombre tabla_final.csv
ruta_final = "/content/drive/MyDrive/Dataset PI/trusted/trusted_data.csv"
shutil.move(archivo_generado, ruta_final)

# Eliminar la carpeta vacía
shutil.rmtree(ruta_salida)

 Tabla final guardada en: /content/drive/MyDrive/Dataset PI/Refined/Refined_temp.csv
